<a href="https://colab.research.google.com/github/nazmulcsembstu/Project1/blob/master/speech_to_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
labels = ["bird", "cat", "dog"]
path = '/content/drive/My Drive/trin/'

In [0]:
import os
import librosa

all_wave = []
all_label = []
for label in labels:
    print(label)
    waves = [f for f in os.listdir(path + '/'+ label) if f.endswith('.wav')]
    for wav in waves:
        samples, sample_rate = librosa.load(path + '/' + label + '/' + wav, sr = 16000)
        samples = librosa.resample(samples, sample_rate, 8000)
        if(len(samples)== 8000) : 
            all_wave.append(samples)
            all_label.append(label)

In [0]:
import numpy as np 

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y=le.fit_transform(all_label)
classes= list(le.classes_)

from keras.utils import np_utils
y=np_utils.to_categorical(y, num_classes=len(labels))
all_wave = np.array(all_wave).reshape(-1,8000,1)

from sklearn.model_selection import train_test_split
x_tr, x_val, y_tr, y_val = train_test_split(np.array(all_wave), np.array(y), stratify=y, test_size = 0.3, random_state=777, shuffle=True)


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, Dropout

model = Sequential()

model.add(Conv1D(filters=8, kernel_size=13, activation='relu',padding='valid', strides=1, input_shape=(8000, 1)))
model.add(MaxPooling1D(3))
model.add(Dropout(0.3))

model.add(Conv1D(filters=16, kernel_size=11, activation='relu', padding='valid', strides=1))
model.add(MaxPooling1D(3))
model.add(Dropout(0.5))

model.add(Conv1D(filters=32, kernel_size=9, activation='relu', padding='valid', strides=1))
model.add(MaxPooling1D(3))
model.add(Dropout(0.5))

model.add(Conv1D(filters=64, kernel_size=7, activation='relu', padding='valid', strides=1))
model.add(MaxPooling1D(3))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(len(labels), activation='softmax'))

model.summary()

In [0]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(x_tr, y_tr ,epochs=50, batch_size=32, validation_data=(x_val,y_val))

In [0]:
def predict(audio):
    prob=model.predict(audio.reshape(1,8000,1))
    index=np.argmax(prob[0])
    return classes[index]

In [0]:
samples, sample_rate= librosa.load(path + 'bird/11.wav', sr = 16000)
samples = librosa.resample(samples, sample_rate, 8000)
print("Text:",predict(samples))

Text: bird
